In [2]:
import threading
import cv2
import mediapipe as mp
import socket
import json

# Define a flag to control when to run the extraction function
run_extraction = False

# Function to extract wrist coordinates using MediaPipe
def extract_wrist_coordinates(client_socket):
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands

    cap = cv2.VideoCapture(0)
    hands = mp_hands.Hands()

    while run_extraction:
        try:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)

            if results.multi_hand_landmarks:
                for landmarks in results.multi_hand_landmarks:
                    wrist_landmark = landmarks.landmark[mp_hands.HandLandmark.WRIST]
                    wrist_coords = {'x': wrist_landmark.x, 'y': wrist_landmark.y, 'z': wrist_landmark.z}
                    print(wrist_coords)
                    json_coords = json.dumps(wrist_coords)
                    client_socket.send((json_coords+ '\n').encode())

            cv2.imshow('MediaPipe Hands', frame)
            if cv2.waitKey(1) == 27:
                break
        except Exception as e:
            print("Error:", e)
            break

    # Release the camera and close OpenCV windows when done
    cap.release()
    cv2.destroyAllWindows()

# Function to start the server socket
def server_socket():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("127.0.0.1", 25001))
    server.listen(1)
    print("Server is listening on port 25001...")

    while True:
        client_socket, addr = server.accept()
        print(f"Connection from {addr[0]}:{addr[1]}")
        
        global run_extraction  # Use the global flag
        
        # Set the flag to True when the connection is open
        run_extraction = True

        # Start the hand thread after the client connects
        hand_thread = threading.Thread(target=extract_wrist_coordinates, args=(client_socket,))
        hand_thread.start()

        # Wait for the hand_thread to finish (connection terminated)
        hand_thread.join()

        # Reset the flag to False when the connection is terminated
        run_extraction = False

# Create and start the socket thread
socket_thread = threading.Thread(target=server_socket)
socket_thread.start()

socket_thread.join()


Server is listening on port 25001...
Connection from 127.0.0.1:63079
{'x': 0.5497680902481079, 'y': 0.5728340148925781, 'z': 5.669607503477891e-07}
{'x': 0.5451505780220032, 'y': 0.6390513181686401, 'z': 4.528412489435141e-07}
{'x': 0.5450435280799866, 'y': 0.839819073677063, 'z': 8.41930557271553e-07}
{'x': 0.5316378474235535, 'y': 0.739253044128418, 'z': 3.046178278509615e-07}
{'x': 0.560099720954895, 'y': 0.8519577980041504, 'z': 5.347093292584759e-07}
{'x': 0.5648036003112793, 'y': 0.8725012540817261, 'z': 6.957263849471929e-07}
{'x': 0.5723459720611572, 'y': 0.8033983111381531, 'z': 7.815739877514716e-07}
{'x': 0.5865224599838257, 'y': 0.7225271463394165, 'z': 6.899641107338539e-07}
{'x': 0.5902173519134521, 'y': 0.6558384895324707, 'z': 6.326479251583805e-07}
{'x': 0.6038654446601868, 'y': 0.5541056990623474, 'z': 7.511214334954275e-07}
{'x': 0.6190882325172424, 'y': 0.45739758014678955, 'z': 5.680198000845849e-07}
{'x': 0.6209694743156433, 'y': 0.4263667166233063, 'z': 4.5713352

In [1]:
import socket

def close_socket_by_port(port):
    try:
        # Create a socket and connect to the localhost on the specified port
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', port))

        # Close the socket connection
        client_socket.close()
        print(f"Socket connection on port {port} closed successfully.")
    except ConnectionRefusedError:
        print(f"No socket connection found on port {port}.")

# Example usage:
# Close a socket connection on port 12345
close_socket_by_port(25001)


No socket connection found on port 25001.
